In [1]:
import numpy as np
import pandas as pd
import sklearn.metrics as metrics

In [2]:
# Get the feature of the training set
def get_features(filename):
    features = pd.read_csv('./'+filename)
    print(features.head(12))
    # train = train.sample(frac=1).reset_index(drop=True)
    # print(train.head())
    return features

# Get the labels of the training set
def get_labels(filename):
    labels = pd.read_csv('./'+filename)
    print(labels.head(12))
    # train = train.sample(frac=1).reset_index(drop=True)
    # print(train.head())
    return labels

# Get the features of the testing set
def get_test_feature(filename):
    test_features = pd.read_csv('./'+filename)
    print(test_features.head(12))
    # train = train.sample(frac=1).reset_index(drop=True)
    # print(train.head())
    return test_features

features = get_features('train_features.csv')
labels = get_labels('train_labels.csv')
test_features = get_test_feature('test_features.csv')
print(features.shape)
print(labels.shape)
print(test_features.shape)

    pid  Time   Age  EtCO2  PTT   BUN  Lactate  Temp  Hgb  HCO3  ...  \
0     1     3  34.0    NaN  NaN  12.0      NaN  36.0  8.7  24.0  ...   
1     1     4  34.0    NaN  NaN   NaN      NaN  36.0  NaN   NaN  ...   
2     1     5  34.0    NaN  NaN   NaN      NaN  36.0  NaN   NaN  ...   
3     1     6  34.0    NaN  NaN   NaN      NaN  37.0  NaN   NaN  ...   
4     1     7  34.0    NaN  NaN   NaN      NaN   NaN  NaN   NaN  ...   
5     1     8  34.0    NaN  NaN   NaN      NaN  37.0  NaN   NaN  ...   
6     1     9  34.0    NaN  NaN   NaN      NaN  37.0  NaN   NaN  ...   
7     1    10  34.0    NaN  NaN   NaN      NaN  37.0  NaN   NaN  ...   
8     1    11  34.0    NaN  NaN  12.0      NaN   NaN  8.5  26.0  ...   
9     1    12  34.0    NaN  NaN  12.0      NaN  38.0  8.5  26.0  ...   
10    1    13  34.0    NaN  NaN   NaN      NaN   NaN  NaN   NaN  ...   
11    1    14  34.0    NaN  NaN   NaN      NaN   NaN  NaN   NaN  ...   

    Alkalinephos   SpO2  Bilirubin_direct  Chloride   Hct  Hear

In [3]:
# Number of NaN in train_features
print(features.shape)
print(features.isnull().sum())

(227940, 37)
pid                      0
Time                     0
Age                      0
EtCO2               218157
PTT                 217641
BUN                 207835
Lactate             217184
Temp                146825
Hgb                 205645
HCO3                215381
BaseExcess          208053
RRate                40155
Fibrinogen          225447
Phosphate           216350
WBC                 208857
Creatinine          210148
PaCO2               206897
AST                 222179
FiO2                201338
Platelets           209905
SaO2                214926
Glucose             180904
ABPm                 32051
Magnesium           210417
Potassium           199547
ABPd                 75522
Calcium             210110
Alkalinephos        222232
SpO2                 32748
Bilirubin_direct    227221
Chloride            214023
Hct                 200643
Heartrate            27812
Bilirubin_total     222614
TroponinI           224164
ABPs                 36290
pH             

In [4]:
# Number of NaN in the test_features
print(test_features.shape)
print(test_features.isnull().sum())

(151968, 37)
pid                      0
Time                     0
Age                      0
EtCO2               145440
PTT                 145115
BUN                 138520
Lactate             144692
Temp                 97813
Hgb                 137096
HCO3                143546
BaseExcess          138730
RRate                26669
Fibrinogen          150313
Phosphate           144282
WBC                 139215
Creatinine          140163
PaCO2               138087
AST                 148060
FiO2                134169
Platelets           140095
SaO2                143298
Glucose             120896
ABPm                 21296
Magnesium           140141
Potassium           132953
ABPd                 50659
Calcium             140076
Alkalinephos        148107
SpO2                 21741
Bilirubin_direct    151490
Chloride            142551
Hct                 133719
Heartrate            18403
Bilirubin_total     148419
TroponinI           149499
ABPs                 23651
pH             

In [7]:
# Delect the columns with too many NaN
for index, row in features.iteritems():
    if features[index].isnull().sum() >= 0.5 * len(row):
        features = features.drop(columns=index)
        test_features = test_features.drop(columns=index)

# Average multiple tests on one patient in the training set
average_features = features.groupby(np.arange(len(features))//12).mean()
print(average_features.shape)

# For the rest columns, check how many are still NaN
for index, row in average_features.iteritems():
    print(average_features[index].isnull().sum())
average_features.head(12)

(18995, 9)
0
0
0
152
107
3882
22
7
398


,pid,Time,Age,RRate,ABPm,ABPd,SpO2,Heartrate,ABPs
0,1.0,8.5,34.0,17.000000,68.333333,50.250000,100.000000,77.083333,114.500000
1,10.0,6.5,71.0,18.090909,101.727273,83.272727,98.000000,78.818182,132.909091
2,100.0,7.5,68.0,14.833333,81.833333,62.833333,96.500000,109.083333,117.000000
3,1000.0,6.5,79.0,12.000000,83.454545,62.818182,98.818182,86.363636,141.909091
4,10000.0,6.5,76.0,12.090909,69.090909,48.227273,98.545455,77.090909,123.000000
5,10002.0,6.5,73.0,19.625000,69.818182,48.500000,99.181818,67.090909,132.090909
6,10006.0,6.5,51.0,18.888889,70.555556,48.714286,96.666667,82.000000,117.888889
7,10007.0,6.5,60.0,21.909091,108.181818,88.363636,94.909091,79.909091,139.363636
8,10009.0,6.5,69.0,22.500000,65.909091,65.227273,97.090909,97.727273,90.909091
9,1001.0,6.5,36.0,13.363636,85.909091,69.181818,100.000000,106.727273,113.090909


In [21]:
# Use KNN to impute data, now the N-neighbors is set to 3(Should be validate later)
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)
imputed = imputer.fit_transform(average_features)
average_features_imputed = pd.DataFrame(imputed, columns=average_features.columns)

# Check now how many are still NaN
for index, row in average_features_imputed.iteritems():
    print(average_features_imputed[index].isnull().sum())
average_features_imputed.head(12)

0
0
0
0
0
0
0
0
0


,pid,Time,Age,RRate,ABPm,ABPd,SpO2,Heartrate,ABPs
0,1.0,8.5,34.0,17.000000,68.333333,50.250000,100.000000,77.083333,114.500000
1,10.0,6.5,71.0,18.090909,101.727273,83.272727,98.000000,78.818182,132.909091
2,100.0,7.5,68.0,14.833333,81.833333,62.833333,96.500000,109.083333,117.000000
3,1000.0,6.5,79.0,12.000000,83.454545,62.818182,98.818182,86.363636,141.909091
4,10000.0,6.5,76.0,12.090909,69.090909,48.227273,98.545455,77.090909,123.000000
5,10002.0,6.5,73.0,19.625000,69.818182,48.500000,99.181818,67.090909,132.090909
6,10006.0,6.5,51.0,18.888889,70.555556,48.714286,96.666667,82.000000,117.888889
7,10007.0,6.5,60.0,21.909091,108.181818,88.363636,94.909091,79.909091,139.363636
8,10009.0,6.5,69.0,22.500000,65.909091,65.227273,97.090909,97.727273,90.909091
9,1001.0,6.5,36.0,13.363636,85.909091,69.181818,100.000000,106.727273,113.090909


In [22]:
# Check the labels
print(labels.shape)
labels.head(12)

(18995, 16)


,pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
0,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,12.1,85.4,100.0,59.9
1,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.8,100.6,95.5,85.5
2,100,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,16.5,88.3,96.5,108.1
3,1000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,19.4,77.2,98.3,80.9
4,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.6,76.8,97.7,95.3
5,10002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.5,67.4,99.1,64.6
6,10006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.2,84.9,96.8,90.5
7,10007,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,23.8,97.0,94.3,76.0
8,10009,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,18.9,72.0,95.3,91.5
9,1001,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,14.9,83.3,99.9,117.3


In [23]:
# Average multiple tests on one patient in the test set
average_test_features = test_features.groupby(np.arange(len(test_features))//12).mean()
print(average_test_features.shape)
average_test_features.head(12)

(12664, 9)


,pid,Time,Age,RRate,ABPm,ABPd,SpO2,Heartrate,ABPs
0,0.0,6.5,39.0,11.714286,80.750000,64.500000,100.000000,84.500000,128.625000
1,10001.0,6.5,62.0,16.666667,91.555556,82.055556,94.200000,111.400000,111.444444
2,10003.0,6.5,72.0,16.818182,82.090909,56.818182,99.636364,91.000000,124.909091
3,10004.0,6.5,44.0,13.857143,66.909091,56.454545,95.727273,87.636364,94.727273
4,10005.0,6.5,88.0,19.272727,71.300000,NaN,95.909091,54.818182,130.100000
5,10008.0,7.5,59.0,18.583333,96.333333,68.166667,98.250000,71.250000,154.000000
6,10011.0,6.5,45.0,14.909091,99.636364,74.727273,99.545455,66.181818,149.090909
7,10017.0,6.5,36.0,22.400000,116.900000,87.800000,98.600000,106.600000,162.700000
8,10018.0,6.5,65.0,17.800000,78.090909,60.454545,94.545455,109.636364,106.000000
9,10019.0,6.5,84.0,15.181818,86.000000,67.045455,96.363636,67.727273,116.181818


In [25]:
# Also impute it using the same KNN imputer
imputed_test_features = imputer.transform(average_test_features)
average_test_features_imputed = pd.DataFrame(imputed_test_features, columns=average_test_features.columns)
average_test_features_imputed.head(12)

,pid,Time,Age,RRate,ABPm,ABPd,SpO2,Heartrate,ABPs
0,0.0,6.5,39.0,11.714286,80.750000,64.500000,100.000000,84.500000,128.625000
1,10001.0,6.5,62.0,16.666667,91.555556,82.055556,94.200000,111.400000,111.444444
2,10003.0,6.5,72.0,16.818182,82.090909,56.818182,99.636364,91.000000,124.909091
3,10004.0,6.5,44.0,13.857143,66.909091,56.454545,95.727273,87.636364,94.727273
4,10005.0,6.5,88.0,19.272727,71.300000,61.242424,95.909091,54.818182,130.100000
5,10008.0,7.5,59.0,18.583333,96.333333,68.166667,98.250000,71.250000,154.000000
6,10011.0,6.5,45.0,14.909091,99.636364,74.727273,99.545455,66.181818,149.090909
7,10017.0,6.5,36.0,22.400000,116.900000,87.800000,98.600000,106.600000,162.700000
8,10018.0,6.5,65.0,17.800000,78.090909,60.454545,94.545455,109.636364,106.000000
9,10019.0,6.5,84.0,15.181818,86.000000,67.045455,96.363636,67.727273,116.181818


In [58]:
# Standardize the training data and import the Logistic Regression model
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Transform the training features and the test features. And the training labels(the first label now)
x_train = average_features_imputed.iloc[:,2:].values
x_test = average_test_features_imputed.iloc[:,2:].values
y_BaseExcess = labels.iloc[:,1:2].values.raval()

# Standardize the training features and test features
ss = StandardScaler()
X_train = ss.fit_transform(x_train)
X_test = ss.transform(x_test)

AttributeError: 'numpy.ndarray' object has no attribute 'raval'

In [53]:
X_train.shape

(18995, 7)

In [54]:
y_BaseExcess.shape

(18995, 1)

In [55]:
X_test.shape

(12664, 7)

In [56]:
# Initialize the LR model
lr = LogisticRegression(C=1.0, penalty='l2', tol=0.001, random_state=2)

In [57]:
lr.fit(X_train, y_BaseExcess)

E:\Anaconda3\envs\intro-ml\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression(random_state=2, tol=0.001)

In [19]:
# # Use KNN to impute the data, optimize the value of K using validation set with the metrics of root mean square 
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error

# rmse = lambda y, yhat: np.sqrt(mean_squared_error(y, yhat))

# def optimize_k(data, target):
#     errors = []
#     for k in range(1, 20, 2):
#         imputer = KNNImputer(n_neighbors=k)
#         imputed = imputer.fit_transform(data)
#         data_imputed = pd.DataFrame(imputed, columns=data.columns)
        
#         # Split the imputed data into training and test set
#         X = data_imputed.drop([''], axis=1)
#         y = data_imputed[target]
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state='imltask2_imputation')

#         model = RandomForestRegressor()
#         model.fit(X_train, y_train)
#         preds = model.predict(X_test)
#         error = rmse(y_test, preds)
#         errors.append({'K': k, 'RMSE': error})
        
#     return errors

In [20]:
# k_errors = optimize_k(data=average_features, target='MEDV')

In [ ]:
# Average multiple tests on one patient in the test set
average_test_features = test_features.groupby(np.arange(len(test_features))//12).mean()
print(average_test_features.shape)
average_test_features.head(12)

In [ ]:
# Standardize the training data and import the Logistic Regression model
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [ ]:
# Transform the training features and the test features. And the training labels(the first label now)
x_train = average_features.iloc[:,2:].values
x_test = average_test_features.iloc[:,2:].values
y_BaseExcess = labels.iloc[:,1:2].values

In [ ]:
x_train

In [ ]:
# Standardize the training features and test features
ss = StandardScaler()
X_train = ss.fit_transform(x_train)
X_test = ss.transform(x_test)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_BaseExcess.shape

In [ ]:
# Initialize the LR model
lr = LogisticRegression(C=1.0, penalty='l2', tol=0.001, random_state='imltask2')

In [ ]:
lr.fit(X_train, y_BaseExcess)